In [ ]:
import sys
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

#sys.path.insert(0, "/usr/src/app/app")
def import_path(fullpath):
    """ 
    Import a file with full path specification. Allows one to
    import from anywhere, something __import__ does not do. 
    """
    path, filename = os.path.split(fullpath)
    filename, ext = os.path.splitext(filename)
    sys.path.append(path)
    module = __import__(filename)
   # reload(module) # Might be out of date
    del sys.path[-1]
    return module

import_path("/usr/src/app/app")

In [ ]:
from PIL import Image

tmp_path = "/usr/src/app/volume/tmp_images"
if not os.path.exists(tmp_path):
    os.mkdir(tmp_path)
def save_image(pil_img:Image.Image, file_name="1.png"):
    im_path = os.path.join(tmp_path, file_name)
    pil_img.save(im_path)


In [ ]:
from app.pkg.ml.try_on.preprocessing.aggregator import ClothProcessor,  BackgroundModels

cp = ClothProcessor(BackgroundModels.SegFormerB3)

In [ ]:
import os
from app.pkg.ml.auto_clothing_set.autoset import LocalRecSys

from app.pkg.ml.buffer_converters import BytesConverter
from PIL import Image
from app.pkg.models.app.image_category import ImageCategory
import random
bc = BytesConverter()


DATA_PATH = "/usr/src/app/data"
def read_clothes(sub_folder):
    
    images_folder = os.path.join(DATA_PATH, sub_folder)
    assert os.path.exists(images_folder)
    im_paths = os.listdir(images_folder)
    abs_im_path = [os.path.join(images_folder, im_path) for im_path in im_paths]
    return abs_im_path


outerwear_clothes_paths = read_clothes("outerwear") # ["/usr/src/app/data/upper/b_bloose.png", "/usr/src/app/data/upper/b_jacket.png"]
upper_clothes_paths = read_clothes("upper") # [ "/usr/src/app/data/upper/t-shirt-blue.png","/usr/src/app/data/upper/t-shirt-miami.png", "/usr/src/app/data/upper/b_t-shirt-2.png", "/usr/src/app/data/upper/b_t-shirt-3.png", "/usr/src/app/data/upper/t-shirt-dc.png", "/usr/src/app/data/upper/t-shirt-dc2.png", "/usr/src/app/data/upper/t-shirt.png" ]
lower_clothes_paths = read_clothes("lower")  # ["/usr/src/app/data/lower/b_black_jeans.png", "/usr/src/app/data/lower/b_shorts.png", "/usr/src/app/data/lower/jeans-dc.png", "/usr/src/app/data/lower/shorts-dc.png" ]
dress_clothes_paths = read_clothes("dresses") # ["/usr/src/app/data/dresses/1.png", "/usr/src/app/data/dresses/2.png", "/usr/src/app/data/dresses/dress-dc.png"]

user_images = [f"{DATA_PATH}/human/brayan_krenston.png"]

def convert_cloth_to_dict(x, category=None):
    res = []
    for i in x:
        image = Image.open(i)
        im_no_back = cp.model_background(image)
        im_bytes = bc.image_to_bytes(im_no_back)
        res.append({'cloth':im_bytes, 'category':category, "clothes_id":random.randint(0, 9999)})
    return res

def convert_user_to_dict(x):
    res = []
    for i in x:
        image = Image.open(i)
        im_no_back = cp.model_background(image)
        im_bytes = bc.image_to_bytes(im_no_back)
        res.append({'image':im_bytes,})
    return res

#        im_white_back = cp.model_background.replace_background_RGBA(im_no_back)


upper_clothes = convert_cloth_to_dict(upper_clothes_paths, ImageCategory.UPPER_BODY)
lower_clothes = convert_cloth_to_dict(lower_clothes_paths, ImageCategory.LOWER_BODY)
dress_clothes = convert_cloth_to_dict(dress_clothes_paths, ImageCategory.DRESSES)
outerwear_clothes = convert_cloth_to_dict(outerwear_clothes_paths, ImageCategory.UPPER_BODY)
user_images = convert_user_to_dict(user_images)


In [ ]:
# save_image(bc.bytes_to_image(lower_clothes[2]['cloth']))

In [ ]:
from typing import Dict, Union, List
import io

import torch
from transformers import AutoProcessor, CLIPModel, AutoTokenizer
from torch.nn import Softmax
from PIL import Image
import numpy as np

from app.pkg.ml.buffer_converters import BytesConverter
from app.pkg.ml.try_on.preprocessing.cloth import ClothPreprocessor

In [ ]:
def sum_normalize(array):
    if isinstance(array, list):
        array = np.array(array)
    return array/array.sum()
# a = [0.1, 0.4, 0.4]
# sum_normalize(a)

In [ ]:
lrs = LocalRecSys(return_cloth_fields=["clothes_id", 'cloth'])
# score не возвращается, т.к это уже часть outfit. return_cloth_fields - для

In [ ]:
lrs.get_max_sets_amount(
            upper_clothes=upper_clothes[:],
            lower_clothes=lower_clothes[:],
            dresses_clothes=dress_clothes[:],
            outerwear_clothes=outerwear_clothes[:],
)

In [ ]:
outfits = lrs.forward(
            upper_clothes=upper_clothes[:],
            lower_clothes=lower_clothes[:],
            dresses_clothes=dress_clothes[:],
            outerwear_clothes=outerwear_clothes[:],
            prompt = "",#'beachwear',# 'clothes for cinema', # "clothes for theatre",#"beachwear",#"clothes for beach", #"party clothes", #
            sample_amount=243
            )

In [ ]:
outfits[0]

In [ ]:
import numpy as np
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

X = torch.concatenate([outfit['tensor'].cpu().unsqueeze(0) for outfit in outfits]).numpy()
#np.array([[0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 1]])

X_embedded = TSNE(n_components=2, learning_rate='auto',
                   init='random', perplexity=3).fit_transform(X)
X_embedded.shape

In [ ]:
from matplotlib import pyplot as plt
kmeans = KMeans(n_clusters=8, random_state=0)
labels = kmeans.fit_predict(X)

# Plot the data with cluster colors
plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=labels)

plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('K-Means Clustering')
plt.grid()
plt.show()

In [ ]:
from plotly import express as px
import pandas as pd

def get_df(data:list, labels):
    return {
     "x":data[:,0],
     "y":data[:,1],
     "label":labels,
     'index':range(len(data))
    #  "uuid":[outfit['uuid'] for outfit in data]
     }



user_df = pd.DataFrame(get_df(X_embedded, kmeans.labels_))

px.scatter(user_df, x='x', y='y', color='label', hover_data=['index']);


In [ ]:
# from matplotlib import pyplot as plt
# plt.rcParams["figure.figsize"] = (20,70)
def plot_outfits(outfits):
 
    fig, axs = plt.subplots(len(outfits), 3)
    for i, outfit in enumerate(outfits):
        #title =  * 3
        # axs[i,0].set_title(f'total: {total_score}')

        # prompt_corr = outfit['prompt_corr']
        # axs[i,1].set_title(f'prompt_corr: {prompt_corr}')

        # clothes_score = outfit['clothes_score']
        # axs[i,2].set_title(f'clothes_score: {clothes_score}')

                           
        for j, cloth in enumerate(outfit['clothes']):
            axs[i, j].imshow(cloth['cloth'])

def plot_outfit(outfits, index):
 
    fig, axs = plt.subplots(1, 3)
    outfit = outfits[index]                   
    for j, cloth in enumerate(outfit['clothes']):
        axs[j].imshow(cloth['cloth'])



# for i in [186, 13]:
#     plot_outfit(outfits, i)


In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (6,3)
import pandas as pd
data = pd.DataFrame({"scores":[outfit['score'] for outfit in outfits]}) # ['scores']
sns.histplot(data=data,
             x="scores",
             kde=True)
